In [39]:
import pandas as pd
import json
from datetime import datetime

#leer base de datos con los partidos
df = pd.read_csv(r"C:\Users\sergi\PROYECTO POWER RANKING\databaseV1.csv")

#Leer df_Teams
with open(r"C:\Users\sergi\PROYECTO POWER RANKING\esports-data\teams.json", 'r') as archivo:
    datos_equipos = json.load(archivo)
df_teams = pd.DataFrame(datos_equipos)

In [40]:
df['Team 1 ID'] = df['Team 1 ID'].astype(str)
df['Team 2 ID'] = df['Team 2 ID'].astype(str)

In [41]:
#se define el df_ranking con los valores iniciales para cada equipo
equipos_unicos = pd.Series(pd.concat([df['Team 1 ID'], df['Team 2 ID']], ignore_index=True).unique())
df_ranking = pd.DataFrame({'TeamID': equipos_unicos})
df_ranking['TeamID'] = df_ranking['TeamID'].astype(str)
df_ranking = df_ranking.merge(df_teams, left_on='TeamID', right_on='team_id', how='left')
df_ranking['score'] = 0
df_ranking = df_ranking[['TeamID','name','score']]


In [42]:
#se ordenan los partidos por fecha, de mas antiguo a mas reciente
df['eventTime'] = pd.to_datetime(df['eventTime'], errors='coerce')  # Utiliza 'coerce' para manejar valores no válidos
df = df.sort_values(by='eventTime')
df.dropna(subset=['eventTime'], inplace=True)
fecha_mas_reciente = df['eventTime'].max()
fecha_un_anio_atras = fecha_mas_reciente - pd.DateOffset(years=1)


# Filtra las filas desde la fecha más reciente hasta un año atrás
df= df[df['eventTime'] >= fecha_un_anio_atras]

In [43]:
#se cambian los win y loss por 1 y 0
# Define un diccionario de mapeo para reemplazar 'win' por 1 y 'loss' por 0
mapeo = {'win': 1, 'loss': 0, 'tie':0}

# Aplica el reemplazo a las columnas 'Outcome 1' y 'Outcome 2'
df['Outcome 1'] = df['Outcome 1'].replace(mapeo)
df['Outcome 2'] = df['Outcome 2'].replace(mapeo)
df.dropna(subset=['Outcome 1', 'Outcome 2'], inplace=True)
df = df.reset_index(drop=True)

In [44]:
#se asigna la importancia segun cada partido
df['mult-key'] = df['region'] + '-' + df['league_name'].astype(str)
mapeo_combinaciones = {
'INTERNATIONAL-MSI':60,
'INTERNATIONAL-Worlds':60,
'CHINA-LPL':50,
'KOREA-LCK':50,
'NORTH AMERICA-LCS':40,
'EMEA-LEC':40,
'HONG KONG, MACAU, TAIWAN-PCS':30,
'VIETNAM-VCS':30,
'JAPAN-LJL':30,
'LATIN AMERICA-LLA':30,
'OCEANIA-LCO':30,
'BRAZIL-CBLOL':30,
'COMMONWEALTH OF INDEPENDENT STATES-LCL':30,
'NORTH AMERICA-LCS Challengers Qualifiers':20,
'EMEA-Greek Legends League':20,
'EMEA-Elite Series':20,
'EMEA-SuperLiga':20,
'EMEA-Ultraliga':20,
'EMEA-TCL':20,
'EMEA-NLC':20,
'JAPAN-LJL Academy':20,
'EMEA-PG Nationals':20,
'EMEA-Esports Balkan League':20,
'LATIN AMERICA SOUTH-South Regional League':20,
'EMEA-Liga Portuguesa':20,
'KOREA-LCK Challengers':20,
'EMEA-Hitpoint Masters':20,
'EMEA-Prime League':20,
'BRAZIL-CBLOL Academy':20,
'EMEA-EMEA Masters':25,
'KOREA-LCK Academy':20,
'EMEA-La Ligue FranÃ§aise':20,
'NORTH AMERICA-LCS Challengers':20,
'EMEA-Arabian League':20,
'LATIN AMERICA NORTH-North Regional League':20,
'NORTH AMERICA-College Championship':20,
'INTERNATIONAL-All-Star Event':20
}

# Reemplazar las combinaciones en la columna 'Combinaciones' por números
df['mult'] = df['mult-key'].replace(mapeo_combinaciones)
df = df.dropna(subset=['mult'], axis=0)
df = df.reset_index(drop=True)
#df.to_csv('df_prueba.csv')

In [45]:
# Bloque principal donde se define la fórmula para la obtención de puntaje

for i in range(len(df)):
    team1_id = df['Team 1 ID'][i]
    team2_id = df['Team 2 ID'][i]
    
    # Verificar si el equipo está en df_ranking antes de acceder a su puntuación
    if team1_id in df_ranking['TeamID'].values:
        score1 = df_ranking.loc[df_ranking['TeamID'] == team1_id, 'score'].values[0]
    else:
        # Crear una nueva fila para el equipo en df_ranking
        new_row = {'TeamID': team1_id, 'name': 'NombreDesconocido', 'score': 0}
        df_ranking = df_ranking.append(new_row, ignore_index=True)
        score1 = 0

    if team2_id in df_ranking['TeamID'].values:
        score2 = df_ranking.loc[df_ranking['TeamID'] == team2_id, 'score'].values[0]
    else:
        # Crear una nueva fila para el equipo en df_ranking
        new_row = {'TeamID': team2_id, 'name': 'NombreDesconocido', 'score': 0}
        df_ranking = df_ranking.append(new_row, ignore_index=True)
        score2 = 0
    
    print(score1,score2)
    
    outcome_1 = df['Outcome 1'][i]
    outcome_2 = df['Outcome 2'][i]

    mult = df['mult'][i]

    dr1 = score1 - score2
    dr2 = score2 - score1
    
    print(dr1,dr2)
    
    We1 = 1 / (10 ** (-dr1 / 600) + 1)
    We2 = 1 / (10 ** (-dr2 / 600) + 1)
    
    print(We1,We2)

    new_score1 = score1 + mult * (outcome_1 - We1)
    new_score2 = score2 + mult * (outcome_2 - We2)

    print(new_score1,new_score2)
    
    df_ranking.loc[df_ranking['TeamID'] == team1_id, 'score'] = new_score1
    df_ranking.loc[df_ranking['TeamID'] == team2_id, 'score'] = new_score2
    
    print(str(team1_id) + " : " + str(new_score1) + " / " + str(team2_id) + " : " + str(new_score2))
    
    if (str(new_score1) == 'nan' or str(new_score2) == 'nan'):
        print(df.index[i])
        break

0 0
0 0
0.5 0.5
-12.5 12.5
105536918475957734 : -12.5 / 105519746234964687 : 12.5
0.0 0.0
0.0 0.0
0.5 0.5
12.5 -12.5
105514907449611976 : 12.5 / 106371141659311452 : -12.5
0.0 0.0
0.0 0.0
0.5 0.5
12.5 -12.5
102787200126663579 : 12.5 / 105553606349477427 : -12.5
0.0 0.0
0.0 0.0
0.5 0.5
-12.5 12.5
99322214689444546 : -12.5 / 105514967858969605 : 12.5
0.0 0.0
0.0 0.0
0.5 0.5
12.5 -12.5
107467803833323099 : 12.5 / 105559480124455983 : -12.5
0.0 0.0
0.0 0.0
0.5 0.5
12.5 -12.5
105504044166670507 : 12.5 / 102787200151698443 : -12.5
12.5 -12.5
25.0 -25.0
0.5239668802413505 0.4760331197586495
24.40082799396624 -24.40082799396624
105514967858969605 : 24.40082799396624 / 105536918475957734 : -24.40082799396624
-12.5 12.5
-25.0 25.0
0.4760331197586495 0.5239668802413505
-24.40082799396624 24.40082799396624
106371141659311452 : -24.40082799396624 / 107467803833323099 : 24.40082799396624
12.5 -12.5
25.0 -25.0
0.5239668802413505 0.4760331197586495
24.40082799396624 -24.40082799396624
1055197462349646

10.0 -10.0
20.0 -20.0
0.5191787948458871 0.4808212051541128
19.61642410308226 -19.616424103082256
105550047979361273 : 19.61642410308226 / 105550036134585264 : -19.616424103082256
10.0 10.0
0.0 0.0
0.5 0.5
0.0 20.0
105550054006182911 : 0.0 / 105550026570060790 : 20.0
-10.0 -10.0
0.0 0.0
0.5 0.5
-20.0 0.0
105550042327365041 : -20.0 / 105550033967461806 : 0.0
0.0 0.0
0.0 0.0
0.5 0.5
-25.0 25.0
99566404581868574 : -25.0 / 100205573496804586 : 25.0
0.0 0.0
0.0 0.0
0.5 0.5
-25.0 25.0
102747101565183056 : -25.0 / 105505619546859895 : 25.0
-47.697452277859966 31.791656755719416
-79.48910903357938 79.48910903357938
0.4243232571360459 0.5756767428639541
-36.18391742058088 20.278121898440332
105559480124455983 : -36.18391742058088 / 105504044166670507 : 20.278121898440332
0.7941109735362044 0.41722421951916644
0.376886754017038 -0.376886754017038
0.5003615890292756 0.49963841097072437
10.78687919295069 -9.57554399989532
107564167676290839 : 10.78687919295069 / 107467794614674479 : -9.57554399989

0.7671517938354881 22.236851885357062
-21.469700091521574 21.469700091521574
0.47941339014483947 0.5205866098551606
21.590616188041913 1.4133874911506368
98926509884398584 : 21.590616188041913 / 103461966951059521 : 1.4133874911506368
10.0 10.0
0.0 0.0
0.5 0.5
20.0 0.0
105593951536470384 : 20.0 / 105594135137147298 : 0.0
-19.910732460942103 -42.721333188802674
22.81060072786057 -22.81060072786057
0.52187076419807 0.47812923580193006
-40.7855630288649 -21.846502620879875
99294153824386385 : -40.7855630288649 / 98767991877340524 : -21.846502620879875
2.8337440729667485 -19.31874999284578
22.15249406581253 -22.15249406581253
0.5212405433700209 0.4787594566299791
21.984122338165914 -38.46912825804495
98767991860392497 : 21.984122338165914 / 98926509885559666 : -38.46912825804495
21.984122338165914 -38.46912825804495
60.453250596210864 -60.453250596210864
0.5577407309208879 0.4422592690791121
39.6744931013304 -56.15949902120943
98767991860392497 : 39.6744931013304 / 98926509885559666 : -56.

39.770055558283666 217.82391187041534
-178.05385631213167 178.05385631213167
0.33552363118968953 0.6644763688103106
22.993873998799188 234.6000934298998
100725845022060229 : 22.993873998799188 / 100205573495116443 : 234.6000934298998
-0.8625169167680582 0.0
-0.8625169167680582 0.8625169167680582
0.4991724930076159 0.5008275069923841
14.162308293003464 -15.024825209771523
104367068120825486 : 14.162308293003464 / 109692074894483834 : -15.024825209771523
0.0 0.0
0.0 0.0
0.5 0.5
-15.0 15.0
108517492697602721 : -15.0 / 106269859169658575 : 15.0
0.0 -15.0
15.0 -15.0
0.5143871841659987 0.48561281583400134
-15.43161552497996 0.43161552497995714
99566406490127625 : -15.43161552497996 / 107700204561086446 : 0.43161552497995714
0.0 0.0
0.0 0.0
0.5 0.5
-15.0 15.0
107760667929003738 : -15.0 / 109692071893352727 : 15.0
-19.616424103082256 0.0
-19.616424103082256 19.616424103082256
0.4811886807381453 0.5188113192618548
-9.240197717845163 -10.376226385237095
107179722233063345 : -9.240197717845163 / 

-8.739952846587576 24.240262616740658
-32.980215463328236 32.980215463328236
0.468400607804148 0.5315993921958521
-32.15998323679498 47.66029300694805
105505619546859895 : -32.15998323679498 / 100205573496804586 : 47.66029300694805
-0.012065331048010108 0.02074927269257465
-0.03281460374058476 0.03281460374058476
0.4999685173261227 0.5000314826738774
14.988879149168309 -14.980195207523748
109668849651338559 : 14.988879149168309 / 101428372607057464 : -14.980195207523748
44.560316300026216 -56.960054205881065
101.52037050590728 -101.52037050590728
0.5961861195543842 0.4038138804456159
26.67473271339469 -39.07447061924954
106269859169658575 : 26.67473271339469 / 107700199633958891 : -39.07447061924954
-15.382993869068441 -17.11789021502051
1.7348963459520679 -1.7348963459520679
0.5016644715448441 0.49833552845515594
-0.43292801541376313 -32.06795606867519
99566406490127625 : -0.43292801541376313 / 109692071893352727 : -32.06795606867519
-30.000357559533235 30.700154757830084
-60.70051231

...

-87.45449325524176 -31.83357340758532
-55.62091984765644 55.62091984765644
0.44683840170599914 0.5531615982940009
-76.39126128936174 -42.89680537346534
103461966971048042 : -76.39126128936174 / 106857761435870473 : -42.89680537346534
35.502542416681976 50.62166406139389
-15.119121644711917 15.119121644711917
0.4854986246608094 0.5145013753391906
25.792569923465788 60.331636554610085
102141671183409133 : 25.792569923465788 / 110428362822825796 : 60.331636554610085
72.81305939465081 -123.23278802017002
196.04584741482083 -196.04584741482083
0.6796913515489645 0.32030864845103546
79.21923236367152 -129.63896098919074
107581765633427097 : 79.21923236367152 / 108319257932089006 : -129.63896098919074
86.00426721362435 -21.190635041712554
107.1949022553369 -107.1949022553369
0.6014176855905309 0.3985823144094692
93.97591350181372 -29.162281329901937
99566405126602853 : 93.97591350181372 / 106857734387052095 : -29.162281329901937
-10.183842174490701 26.460561164861616
-36.644403339352316 36.64

-69.85923628645467 112.5088598600813
-182.36809614653598 182.36809614653598
0.33184250725401054 0.6681574927459895
-76.49608643153489 119.14571000516152
105503948999800557 : -76.49608643153489 / 107416483509696259 : 119.14571000516152
49.33739464539642 43.31610148391205
6.0212931614843725 -6.0212931614843725
0.5057766345757256 0.4942233654242743
59.22186195388191 33.43163417542657
105515345386578249 : 59.22186195388191 / 107450819235010833 : 33.43163417542657
11.660840092823358 -33.10670458465745
44.76754467748081 -44.76754467748081
0.5428451180266884 0.4571548819733115
0.8039377322895884 -22.24980222412368
105537149834356301 : 0.8039377322895884 / 105536738266035235 : -22.24980222412368
77.35759136772688 24.20029765361425
53.15729371411263 -53.15729371411263
0.5508235318162139 0.4491764681837861
66.34112073140261 35.21676828993853
105520796734358045 : 66.34112073140261 / 107128093558481418 : 35.21676828993853
66.75248468519247 -74.49405946607348
141.24654415126594 -141.24654415126594


-77.57463603519199 40.163646132055646
-117.73828216724763 117.73828216724763
0.3889240071900396 0.6110759928099604
-85.35311617899278 47.94212627585644
107565448721890834 : -85.35311617899278 / 109642680932009857 : 47.94212627585644
12.67447822026863 45.307037487778615
-32.63255926750998 32.63255926750998
0.4687328345429751 0.5312671654570249
23.29982152940913 34.68169417863812
110418022381234710 : 23.29982152940913 / 107597870621708485 : 34.68169417863812
-28.99733493102655 119.17434226711387
-148.17167719814043 148.17167719814043
0.3615530456089999 0.6384469543910001
-36.22839584320655 126.40540317929387
107599485973747723 : -36.22839584320655 / 109800832177252614 : 126.40540317929387
-96.04435630143985 -2.797446514956171
-93.24690978648368 93.24690978648368
0.41148054579708243 0.5885194542029175
-104.2739672173815 5.432164400985478
109696138543579708 : -104.2739672173815 / 109696115986547257 : 5.432164400985478
131.08822750958203 62.07580565513389
69.01242185444814 -69.0124218544481

-75.60755742488169 -91.84218221995057
16.234624795068882 -16.234624795068882
0.5155706324830748 0.48442936751692517
-61.074676399373935 -106.37506324545834
107251295460371069 : -61.074676399373935 / 107258254469504696 : -106.37506324545834
-72.21244104079159 -145.8839264338935
73.6714853931019 -73.6714853931019
0.5702141114652985 0.42978588853470134
-89.31886438475054 -128.77750308993453
109692071893352727 : -89.31886438475054 / 109692074894483834 : -128.77750308993453
68.87146801862895 72.53175152267788
-3.660283504048934 3.660283504048934
0.4964883434778849 0.503511656522115
53.976817714292395 87.42640182701443
107760667929003738 : 53.976817714292395 / 106269804045322803 : 87.42640182701443
122.36774186436632 99.42757721184697
22.940164652519357 -22.940164652519357
0.5219948299949383 0.47800517000506176
136.70789696451817 85.08742211169512
98767991954244555 : 136.70789696451817 / 108573042084687884 : 85.08742211169512
250.2013094301913 14.214177967293669
235.98713146289762 -235.98713

-5.02419379881246 118.88529825104509
-123.90949204985755 123.90949204985755
0.38331050975255004 0.6166894902474499
-14.606956542626211 128.46806099485883
105593182576105603 : -14.606956542626211 / 103935468920814040 : 128.46806099485883
102.34299118597856 55.00049193009279
47.34249925588577 -47.34249925588577
0.5452963586308743 0.4547036413691257
113.7105822202067 43.63290089586465
105520796734358045 : 113.7105822202067 / 105594135137147298 : 43.63290089586465
51.87984044052145 103.72143097910627
-51.841590538584825 51.841590538584825
0.45042604333968933 0.5495739566603106
65.61918935702921 89.98208206259851
109696092218588987 : 65.61918935702921 / 108352305932141947 : 89.98208206259851
161.04775752061767 57.24235738018073
103.80540014043694 -103.80540014043694
0.5982954782560844 0.40170452174391563
171.09037056421556 47.19974433658284
105593951536470384 : 171.09037056421556 / 109642680932009857 : 47.19974433658284
158.79246811946877 178.91072302193862
-20.118254902469857 20.1182549024

In [46]:
#se ordena el ranking por puntaje
df_ranking = df_ranking.sort_values(by='score', ascending=False)
df_ranking = df_ranking.reset_index(drop=True)
df_ranking['index'] = df_ranking.index
df_ranking.to_csv('ranking.csv', index=False)

In [47]:
df_ranking.to_json('ranking.json', orient='records', lines=False)